In [47]:
import os
import torch
import torch.utils.data
import pandas as pd
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset



In [132]:
CUDA = False
SEED = 1
BATCH_SIZE = 1
LOG_INTERVAL = 10
EPOCHS = 10
VALIDATION_SPLIT = .2
ZDIMS = 20

In [49]:
torch.manual_seed(SEED)

kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}

In [27]:
#train_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
#    batch_size=BATCH_SIZE, shuffle=True, **kwargs

#)

#test_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
#    batch_size=BATCH_SIZE, shuffle=True, **kwargs)               




In [12]:
import pandas as pd

kmer_path = "data\kmers-gzip\\output.txt.gz"

kmer_all = pd.read_csv(kmer_path ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"], index_col = 0)

#kmers = kmer_all.set_index("kmer", drop=False)




In [13]:
kmer_culled = kmer_all.loc[kmer_all['count'] >= 700].copy()
print(kmer_all.describe())

print(kmer_culled.describe())




              count
count  3.388926e+07
mean   5.418934e+01
std    9.312541e+01
min    2.000000e+00
25%    3.000000e+00
50%    1.000000e+01
75%    5.200000e+01
max    8.817000e+03
             count
count  8763.000000
mean    923.065617
std     320.572167
min     700.000000
25%     780.000000
50%     851.000000
75%     968.000000
max    8817.000000


In [40]:

kmer_path = "data\kmers-gzip\\s314.txt.gz"
kmer_path2 = "data\kmers-gzip\\upec-261.txt.gz"


kmer_contig1 = pd.read_csv(kmer_path ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"], index_col = 0)
kmer_contig2 = pd.read_csv(kmer_path2 ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"], index_col = 0)


kmer_contig1.sort_values(by="kmer", inplace=True)


contig1_parsed = kmer_contig1[kmer_contig1.index.isin(kmer_culled.index)]
contig2_parsed = kmer_contig2[kmer_contig2.index.isin(kmer_culled.index)]


kmer_temp = kmer_culled.copy()
kmer_temp['count'] = '0'
kmer_temp.sort_values(by='kmer', inplace=True)
print(contig1_parsed)
print(kmer_temp.head(70))
print(kmer_temp.loc["AAAAACTCTGCTTACCAGGCGCATTTCGCCC"])

                                 count
kmer                                  
AAAAAACTCAGGCCGCAGTCGGTAACCTCGC      4
AAAAAAGAATAAAACTCAGTCTTGGGGGACT     10
AAAAACACCATCATACACTAAATCAGTAAGT     13
AAAAACATGATCACCGGTGCTGCTCAGATGG      3
AAAAACCTCCGGCTATGCCGGAGGATATTTA     26
...                                ...
TTTTGATTTCTTGTTTGACACTTGCAGTTGC     26
TTTTGCCGTTACGCACCACCCCGTCAGTAGC     12
TTTTGTAGGCCGGATAAGGCGTTCACGCCGC      3
TTTTGTCTTATTCAAAGGCCTTACATTTCAA      9
TTTTTGAAATGTAAGGCCTTTGAATAAGACA      9

[2856 rows x 1 columns]
                                count
kmer                                 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA     0
AAAAAAAGCGCCAGGAGCTGGAGAAAAAACG     0
AAAAAACTCAGGCCGCAGTCGGTAACCTCGC     0
AAAAAAGAATAAAACTCAGTCTTGGGGGACT     0
AAAAAAGCGCCAGGAGCTGGAGAAAAAACGA     0
...                               ...
AAAAGCGCCAGGAGCTGGAGAAAAAACGAAA     0
AAAAGCTCGTGGTATCACCATCAACACTTCT     0
AAAAGGCTACCGCAAATCTGGTAATGAGGCC     0
AAAAGGCTGCCTCATCGCTAACTTTGCAACA     0
AAAAGGGGATGA

In [41]:
contig1_merged = pd.merge(contig1_parsed, kmer_temp, how='right',on='kmer')
contig2_merged = contig2_parsed.merge(kmer_temp, how='right',on='kmer')

contig_noNull = contig1_merged[[ 'count_x']].fillna(value='0').copy()
contig_noNull2 = contig2_merged[[ 'count_x']].fillna(value='0').copy()
pd.isnull(contig_noNull)


contig_noNull.sort_values(by=['kmer'], inplace=True)
contig_noNull2.sort_values(by=['kmer'], inplace=True)

#print(kmer_temp.describe())

print(contig_noNull.head(10))
print(contig_noNull2.head(10))
#contig_clean = contig1_merged[['kmer', 'count_x']].copy()
#contig_clean.head()
#pd.isnull(contig_clean)


                                count_x
kmer                                   
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA       0
AAAAAAAGCGCCAGGAGCTGGAGAAAAAACG       0
AAAAAACTCAGGCCGCAGTCGGTAACCTCGC       4
AAAAAAGAATAAAACTCAGTCTTGGGGGACT      10
AAAAAAGCGCCAGGAGCTGGAGAAAAAACGA       0
AAAAACACAGCACTGTGCAAACACGAAAGTG       0
AAAAACACCATCATACACTAAATCAGTAAGT      13
AAAAACATGATCACCGGTGCTGCTCAGATGG       3
AAAAACCTCCGGCTATGCCGGAGGATATTTA      26
AAAAACGGCTGCGCTGGTACTGGCGTAACCC       0
                                count_x
kmer                                   
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA       0
AAAAAAAGCGCCAGGAGCTGGAGAAAAAACG       4
AAAAAACTCAGGCCGCAGTCGGTAACCTCGC       0
AAAAAAGAATAAAACTCAGTCTTGGGGGACT       4
AAAAAAGCGCCAGGAGCTGGAGAAAAAACGA       4
AAAAACACAGCACTGTGCAAACACGAAAGTG       7
AAAAACACCATCATACACTAAATCAGTAAGT       4
AAAAACATGATCACCGGTGCTGCTCAGATGG       2
AAAAACCTCCGGCTATGCCGGAGGATATTTA       0
AAAAACGGCTGCGCTGGTACTGGCGTAACCC       6


In [167]:


CULL_SIZE = 700

class KmerDataset(Dataset):
    
    def __init__(self, dirname):
        files = os.listdir(dirname)
        max_kmer = 0
        X, y = [],[]
        kmer_all = pd.read_csv('data\kmers-gzip\\output.txt.gz' ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"])
        kmer_culled = kmer_all.loc[kmer_all['count'] >= CULL_SIZE]
        kmer_temp = kmer_culled.copy()
        kmer_temp['count'] = '0'
        kmer_temp.sort_values(by=['kmer'], inplace=True)
        self.template = kmer_temp
        for line in files:
            if(line != 'output.txt.gz'):

                kmer_path = dirname + '\\' + line
                kmer_df = pd.read_csv(kmer_path ,delim_whitespace=True, compression="gzip", names = ["kmer", "count"])
                contig_parsed = kmer_df[kmer_df.kmer.isin(self.template.kmer)]
                contig_merged = contig_parsed.merge(self.template, how='right',on='kmer')

                contig_final = contig_merged[['kmer', 'count_x']].fillna(value=0).copy()
                contig_final.sort_values(by=['kmer'], inplace=True)
                contig_vec = torch.tensor(contig_final['count_x'].values, dtype=torch.float)
                X.append(contig_vec)
                contig_max = torch.max(contig_vec)
                if(contig_max > max_kmer): max_kmer = contig_max
                
        self.X = X
        self.max_kmer = max_kmer
    
    def preprocess(self, contig):
        
        norm_vec = torch.div(contig, self.max_kmer)

        #print(contig_final.head(10))
        
        #print(np.isnan(contig_vec))
        
        return norm_vec
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, index):
        
        return self.preprocess(self.X[index])

In [168]:
#initialize dataset
dataset = KmerDataset('data\kmers-gzip')



In [169]:
dataset[100]

tensor([0.0000, 0.0000, 0.0357,  ..., 0.0000, 0.0397, 0.0000])

In [170]:
#Separate into train/val

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
np.random.seed(SEED)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

#initialize data loaders

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                                sampler=valid_sampler)
print(split)

69


In [171]:
len(test_loader)

69

In [155]:
for i in enumerate(test_loader):
    
    print('index ' + str(i[0]) + 'data: ' + str(i[1]) )

index 0data: tensor([[0.0000, 0.0079, 0.0000,  ..., 0.0079, 0.0198, 0.0079]],
       dtype=torch.float64)
index 1data: tensor([[0.0000, 0.0238, 0.0000,  ..., 0.0238, 0.0000, 0.0238]],
       dtype=torch.float64)
index 2data: tensor([[0.0000, 0.0000, 0.0119,  ..., 0.0000, 0.0278, 0.0000]],
       dtype=torch.float64)
index 3data: tensor([[0.0000, 0.0000, 0.0079,  ..., 0.0000, 0.0278, 0.0000]],
       dtype=torch.float64)
index 4data: tensor([[0.0000, 0.0000, 0.0238,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)
index 5data: tensor([[0.0000, 0.0000, 0.0278,  ..., 0.0000, 0.0159, 0.0000]],
       dtype=torch.float64)
index 6data: tensor([[0.0000, 0.0000, 0.0079,  ..., 0.0000, 0.0317, 0.0000]],
       dtype=torch.float64)
index 7data: tensor([[0.0000, 0.0119, 0.0000,  ..., 0.0119, 0.0000, 0.0119]],
       dtype=torch.float64)
index 8data: tensor([[0.0000, 0.0119, 0.0000,  ..., 0.0119, 0.0000, 0.0119]],
       dtype=torch.float64)
index 9data: tensor([[0.0000, 0.0000, 0.0000, 

In [172]:

#Variable is deprecated update at some point

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(8763, 400)
        
        self.relu = nn.ReLU()
        self.fc21 = nn.Linear(400, ZDIMS)
        self.fc22 = nn.Linear(400, ZDIMS)
        
        
        self.fc3 = nn.Linear(ZDIMS, 400)
        
        self.fc4 = nn.Linear(400, 8763)
        self.sigmoid = nn.Sigmoid()
        
    
    #if in training creates random vector based on mean and stddev
    #if in otherwise returns constant mean 
    def reparameterize(self, mu: Variable, logvar: Variable) -> Variable:
        
        if self.training:
            
            std = logvar.mul(.5).exp_()
            
            eps = Variable(std.data.new(std.size()).normal_())
            
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def encode(self, x: Variable) -> (Variable, Variable):
        
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def decode(self, z: Variable) -> Variable:
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x: Variable) -> (Variable, Variable, Variable):
        #feed forward for the network
        
        #encodes data into mean(mu) and varience(logvar)
        mu, logvar = self.encode(x.view(-1, 8763))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

        

In [173]:
model = VAE()
if CUDA:
    model.cuda()


def loss_function(recon_x, x, mu, logvar) -> Variable:
    
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 8763))
    
    
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    KLD /= BATCH_SIZE * 8763
    
    return BCE + KLD



In [177]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train(epoch):
    
    model.train()
    train_loss = 0
    
    for i in enumerate(train_loader):
        data = i[1]
        batch_idx = i[0]
        data = Variable(data)
        if CUDA:
            data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, logvar = model(data)
        
        loss = loss_function(recon_batch, data, mu, logvar)
        
        loss.backward()
        
        train_loss += loss.data.item()
        optimizer.step()
        
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len (train_loader.dataset),
            100. * batch_idx / len(train_loader),
            loss.data.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)
    ))

In [178]:
def test(epoch):
    
    model.eval()
    test_loss = 0
    
    for i in enumerate(test_loader):
        data = i[1]
        if CUDA:
            data = data.cuda()
        
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data.item()
        
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [181]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)
    
    
    sample = Variable(torch.randn(64, ZDIMS))
    
    if CUDA:
        sample = sample.cuda()
    
    sample = model.decode(sample).cpu()
    
    
    #save_image(sample.data.view(64, 1, 28, 28),
              #'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/347 (0%)]	Loss: 0.078007
Train Epoch: 1 [10/347 (4%)]	Loss: 0.033155
Train Epoch: 1 [20/347 (7%)]	Loss: 0.039076
Train Epoch: 1 [30/347 (11%)]	Loss: 0.035656
Train Epoch: 1 [40/347 (14%)]	Loss: 0.035119
Train Epoch: 1 [50/347 (18%)]	Loss: 0.035899
Train Epoch: 1 [60/347 (22%)]	Loss: 0.072651
Train Epoch: 1 [70/347 (25%)]	Loss: 0.115288
Train Epoch: 1 [80/347 (29%)]	Loss: 0.068152
Train Epoch: 1 [90/347 (32%)]	Loss: 0.021965
Train Epoch: 1 [100/347 (36%)]	Loss: 0.045002
Train Epoch: 1 [110/347 (40%)]	Loss: 0.066005
Train Epoch: 1 [120/347 (43%)]	Loss: 0.070047
Train Epoch: 1 [130/347 (47%)]	Loss: 0.029782
Train Epoch: 1 [140/347 (50%)]	Loss: 0.032460
Train Epoch: 1 [150/347 (54%)]	Loss: 0.035219
Train Epoch: 1 [160/347 (58%)]	Loss: 0.055389
Train Epoch: 1 [170/347 (61%)]	Loss: 0.039869
Train Epoch: 1 [180/347 (65%)]	Loss: 0.068711
Train Epoch: 1 [190/347 (68%)]	Loss: 0.057719
Train Epoch: 1 [200/347 (72%)]	Loss: 0.029238
Train Epoch: 1 [210/347 (76%)]	Loss: 0.051707
Tr

C:\Users\Jacob\Anaconda3\envs\mGWAS\lib\site-packages\ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()



Train Epoch: 2 [0/347 (0%)]	Loss: 0.039856
Train Epoch: 2 [10/347 (4%)]	Loss: 0.029749
Train Epoch: 2 [20/347 (7%)]	Loss: 0.033651
Train Epoch: 2 [30/347 (11%)]	Loss: 0.036625
Train Epoch: 2 [40/347 (14%)]	Loss: 0.067918
Train Epoch: 2 [50/347 (18%)]	Loss: 0.033351
Train Epoch: 2 [60/347 (22%)]	Loss: 0.077803
Train Epoch: 2 [70/347 (25%)]	Loss: 0.031333
Train Epoch: 2 [80/347 (29%)]	Loss: 0.056931
Train Epoch: 2 [90/347 (32%)]	Loss: 0.086439
Train Epoch: 2 [100/347 (36%)]	Loss: 0.067550
Train Epoch: 2 [110/347 (40%)]	Loss: 0.024527
Train Epoch: 2 [120/347 (43%)]	Loss: 0.067019
Train Epoch: 2 [130/347 (47%)]	Loss: 0.030009
Train Epoch: 2 [140/347 (50%)]	Loss: 0.060430
Train Epoch: 2 [150/347 (54%)]	Loss: 0.032607
Train Epoch: 2 [160/347 (58%)]	Loss: 0.066015
Train Epoch: 2 [170/347 (61%)]	Loss: 0.081612
Train Epoch: 2 [180/347 (65%)]	Loss: 0.038139
Train Epoch: 2 [190/347 (68%)]	Loss: 0.042660
Train Epoch: 2 [200/347 (72%)]	Loss: 0.060631
Train Epoch: 2 [210/347 (76%)]	Loss: 0.021420
T

Train Epoch: 8 [40/347 (14%)]	Loss: 0.033140
Train Epoch: 8 [50/347 (18%)]	Loss: 0.077920
Train Epoch: 8 [60/347 (22%)]	Loss: 0.057592
Train Epoch: 8 [70/347 (25%)]	Loss: 0.031187
Train Epoch: 8 [80/347 (29%)]	Loss: 0.092030
Train Epoch: 8 [90/347 (32%)]	Loss: 0.043460
Train Epoch: 8 [100/347 (36%)]	Loss: 0.028127
Train Epoch: 8 [110/347 (40%)]	Loss: 0.065867
Train Epoch: 8 [120/347 (43%)]	Loss: 0.115663
Train Epoch: 8 [130/347 (47%)]	Loss: 0.032034
Train Epoch: 8 [140/347 (50%)]	Loss: 0.031380
Train Epoch: 8 [150/347 (54%)]	Loss: 0.052352
Train Epoch: 8 [160/347 (58%)]	Loss: 0.043095
Train Epoch: 8 [170/347 (61%)]	Loss: 0.023582
Train Epoch: 8 [180/347 (65%)]	Loss: 0.030786
Train Epoch: 8 [190/347 (68%)]	Loss: 0.065708
Train Epoch: 8 [200/347 (72%)]	Loss: 0.087520
Train Epoch: 8 [210/347 (76%)]	Loss: 0.024732
Train Epoch: 8 [220/347 (79%)]	Loss: 0.038753
Train Epoch: 8 [230/347 (83%)]	Loss: 0.025223
Train Epoch: 8 [240/347 (86%)]	Loss: 0.027710
Train Epoch: 8 [250/347 (90%)]	Loss: 0.0